# Jugando a salvar a Jack en el Titanic
Vamos a trabajar un problema sencillo de **regresión** (tomar una decisión de acuerdo a un patrón en los datos) el cual se basa en definir *quien se salva o no* en el evento catastrófico del **Titanic**); este problema se encuentra en Kaggle[https://www.kaggle.com/competitions/titanic].

Se instalan paquetes de python útiles para resolver este problema, tanto para ML como para análisis y visualización de datos.

In [ ]:
# instalando paquetes
!pip install pyarrow
!pip install pandas
!pip install sklearn==0.0
!pip install scikit-learn
!pip install scikit-optimize
!pip install seaborn
!pip install xgboost

PIP es el manejador central de paquetes de Python, aunque hay herramientas lindas para manejar entornos virtuales y paquetes (recomiendo que chequeen ___PyEnv___ y ___Poetry___).

In [ ]:
# verificando paquetes
!pip freeze

Ahora si es el momento de empezar a programar, pero para efectos prácticos, vamos a importar las *funcionalidades* que vamos a utilizar de los **paquetes** que hemos instalado. Esto es muy sencillo.

In [ ]:
# importando lo necesario
import pandas as pd
import seaborn as sns
import xgboost as xgb
from numpy import absolute
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from skopt import BayesSearchCV

## Preparando el conjunto de datos
Preparar el conjunto de datos implica cargarlos en la memoria, quizás hacer algunos procesamientos y análisis para entender los datos, definir posibles limpiezas, y así brindar el mejor conocimiento posible al modelo para su entrenamiento (también se puede hacer *Feature Engineering*).

En primera instancia, vamos a tomar los datos que están en un archivo **CSV** (*Colon Separated Values*) y lo vamos  cargar en un *Pandas DataFrame* (es una forma eficiente de cargar los datos de forma tabular en la **memoria**).

In [ ]:
# la información se coloca en un dataframe (tabla)
raw_data = pd.read_csv('train.csv')
# vamos a imprimir los nombres de las columnas y las dimensiones del dataframe
print(raw_data.columns, raw_data.shape)
# vamos a ver las tres primeras filas cargadas para verificar que la información se haya cargado correctamente
print(raw_data.head(3))

Una de las cosas más simples a verificar es que *columnas no aportan información* para detectar **patrones**, es decir, datos secuenciales, aletarorios, o relacionados con IDs, entre otros.

In [ ]:
# ejemplo de una columna que no permite encontrar un patrón
ticket_list = raw_data['Ticket']
print(ticket_list)

Hay algunas columnas que si pueden aportar información por la naturaleza de lo que significan, pero otro problema es el de **datos faltantes**; cuando la columna es númerica y es posible generar el valor a partir de otras columnas se suelen hacer procesos de auto-completador de información (como el uso de *Imputers*). Hay formas de verificar eso, por ejemplo, contar los valores no númericos presentes en una columna numérica.

In [ ]:
# muchas celdas sin valor....no sirven para enseñar
nan_cabin = raw_data['Cabin'].isna().sum()
print("Cabin stats: ", nan_cabin, (nan_cabin / raw_data.shape[0]))
print(raw_data.info())

Es normal que del conjunto original de datos solo se **seleccionen** algunas caraterísticas (***features***). Esto quiere decir que se hace un listado de las columnas que se quieren usar para el entrenamiento. De igual forma, es necesario **separar** la columna que tiene guarda la **salida**, para hacer correctamente la separación de los conjuntos de datos de entrenamiento y validación.

In [ ]:
# Primera selección de potenciales features, dsecartando las columnas de Ticket, PassengerId, & Cabin
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = ['Survived'] # columna que guarda la salida que interesa aprender a predecir
print(raw_data[features].head(3)) # verificar cómo queda el conjunto de datos con la selección inicial de features

Hay *features* **numéricas** y **categóricas** (no numéricas, cualitativas, texto, similares). En general, las variables *categóricas* se *codifican a* valores *numéricos* para facilitar el proceso de aprendizaje: el procesamiento y detección de patrones sobre valores numéricos suele ser más eficiente (*recursos*, *calidad*) en conjuntos de datos numéricos.

In [ ]:
# Convert Sex from Categorical to Numerical
encoder_sex = LabelEncoder() # Traído de la librería de Scikit-Learn
encoded_sex = encoder_sex.fit(raw_data['Sex'])
raw_data['Sex'] = encoded_sex.transform(raw_data['Sex'])

# Convert Embarked from Categorical to Numerical
encoder_embarked = LabelEncoder() # Traído de la librería de Scikit-Learn
encoded_embarked = encoder_embarked.fit(raw_data['Embarked'])
raw_data['Embarked'] = encoded_embarked.transform(raw_data['Embarked'])
print(raw_data.head(3))

Otra cosa útil en el momento de querer comprender los datos es determinar ***features*** **redundantes** (como edad y fecha de nacimiento), una forma de observar esto es determinar la correlación entre los features que se tienen para el entrenamiento.

In [ ]:
# Dibujar realaciones entre las variables
data_ = raw_data[target + features] # subconjunto de los datos originales usando los features seleccionados hasta el momento
# Se usa Seaborn PAIRPLOR: dibuja relación de las variables presentadas en el conjunto de datos 
plot = sns.pairplot(data_, hue='Survived', diag_kind='kde', corner=True) # La salida ayuda a ver la influencia de las features
plot.map_lower(sns.kdeplot, levels=2, color=".2") 

# Se define el conjunto de datos definitivo a utilizar como conocimiento para el modelo 
features_to_remove = ['Fare', 'SibSp']
features = list(set(features)-set(features_to_remove))
data = raw_data[features]
print(data.head(3))
print(data_.corr())

## Haciendo Machine Learning

Teniendo el conjunto de datos definitivo, se procese a hacer la división del conjunto de datos de entrenamiento (prefijo *train*) y el conjunto de datos de validación (prefijo *test*). En ambos casos, se separa un dataset de *features* (sufijo *_X*) y un dataset de *output* (sufijo *_y*).

In [ ]:
# se usa un función de Scikit-Learn para tomar los datos y generar los subconjuntos correspondientes
train_X, test_X, train_y, test_y = train_test_split(data, raw_data[target], test_size=0.25)
print("Train features:", train_X.shape)
print("Validation features:", test_X.shape)
print("Train target:", train_y.shape)
print("Validation target:", test_y.shape)

Vamos a entrenar un modelo de *Machine Learning* de **clasificación** (usar datos del pasado para determinar **patrones** de comportamiento, predicción). En este caso se va a utilizar **XGBoost**, un algoritmo basado en *Gradient Boosting* y *Random Forest*, que tiene un muy buen desempeño (bajo consumo de recursos y buen nivel de precisión) en problemas de clasificación y regresión sobre data estructurada (en especial aquella con alto componente numérico). 

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='hist',
                          grow_policy='lossguide')
model.fit(train_X, train_y, verbose=False)
print(model)
#print(model.best_iteration)
#print(model.best_score)
print(model.feature_names_in_)
print(model.feature_importances_)

Para validar el modelo se suele calcular algún tipo de error, entendiendo que en *Inteligencia  Artificial* entre menor el error mejor se considera el aprendizaje.
*Cross-validation* es una técnica de segmentación de datos para validar la eficacia del modelo intentando no caer en sesgos.
El error medio absoluto (**MAE** - *Mean Absolute Error*) es un tipo de error muy utilizado en problemas de clasificación para determinar en promedio cuantas veces falla el modelo con respecto al resultado esperado.

In [ ]:
# Se define un modelo de validación del modelo
cross_validation = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, test_X, test_y, scoring='neg_mean_absolute_error', cv=cross_validation, n_jobs=-1)

# Para evitar malas interpretaciones, se convierte a positivo el valor del error
scores = absolute(scores)
print(f"Valores del MAE: Media {scores.mean()}. Varianza: {scores.std()}")

Ahora si, a hacer predicciones a ver qué pasa...para ello primero vamos a cargar los datos del archivo de *test* para determinar su composición, cambiar las variables categóricas a numéricas según corresponde, y trabajar únicamente con los *features* que se definieron para el modelo.

In [ ]:
# Se carga en un dataframe de Pandas la información de pruebas
test_data = pd.read_csv('test.csv')
test_data = test_data[features] # Solamente se quedan las columnas utilizadas en el modelo de ML

# Utilizando los codificadores definidos en el entrenamiento, se convierten las variables categóricas
test_data['Sex'] = encoded_sex.transform(test_data['Sex'])
test_data['Embarked'] = encoded_embarked.transform(test_data['Embarked'])

print(test_data.head()) # Verificación del dataframe luego de las transformaciones realizadas

Se utiliza el modelo entrenado utilizando XGBoost, y se generan las predicciones usando el conjunto de datos *test* como parámetro directo. Esto va a retornar una lista con los valores de *0* ó *1* para cada uno de los pasajeros  evaluar, es decir, arrojará el resultado de la clasificación entre no sobrevivir o sobrevivir.

In [ ]:
# Generar predicciones del modelo
output = model.predict(test_data)
print("Sobrevivientes:", output)

Vamos a ajustar la salida del modelo en un dataframe en donde se va asignar a cada *PassengerId* la salida obtenida del modelo, y así vamos a tener un archivo para enviar a evaluar en la competencia, y tener una forma bonita de comprobar cosas.

In [ ]:
# copiar datos a un nuevo dataframe
full_ouput = test_data.copy()
# agregar la salida del modelo en una columna
full_ouput['Survived'] = output
# generar salida para ver la distribución de variables
output_plot = sns.PairGrid(full_ouput, hue='Survived')
output_plot.map_upper(sns.histplot)
output_plot.map_lower(sns.kdeplot, fill=True)
output_plot.map_diag(sns.histplot, kde=True)
# convertir de nuevo a variables categóricas
full_ouput['Sex'] = encoded_sex.inverse_transform(full_ouput['Sex'])
full_ouput['Embarked'] = encoded_embarked.inverse_transform(full_ouput['Embarked'])
# verificar dataframe
print(full_ouput.info())
print(full_ouput.head())

Vamos a ver un poco de los sobrevivientes en el archivo de pruebas, a ver si nos parece coherente.

In [ ]:
# filtrar el dataframe solo por los sobrevivientes
test_survived = full_ouput[full_ouput['Survived']==1]
print(test_survived)

Se puede hacer una comparación con la salida esperada si solo se salvaran mujeres, para ello usaremos la medida de *accuracy error*, y así contabilisan las diferencias entre la salida esperada contra la salida que obtuvimos en el modelo (es normal que hayan diferencias).

In [ ]:
gender_data = pd.read_csv("gender_submission.csv")
accuracy = accuracy_score(gender_data['Survived'], full_ouput['Survived'])
print(f"Accuracy: {accuracy * 100}")

Utilizando el conjunto de datos de validación podemos intentar encontrar otros errores, por intentar medir potenciales mejoras del modelo, entender mejor su comportamiento, o simplemente tener un punto de comparación en diferentes ejecuciones del modelo.

**Log Loss**: Usado en modelos de clasificación binaria. Calcula el logaritmo de la diferencia entre la probabilidad esperada (0 ó 1) con las probabilidades obtenidas. Más cerca a *cero* es mejor.

**ROC AUC**: Tipo de error comúnmente usado en problemas de clasificación. Determina que tan bueno es un modelo para distinguir las clases finales. Más cerca a *uno* es mejor, y más cercano a *cero* es peor.

In [ ]:
# se calculan las probabilidades y no el grupo de clasificación
output_test = model.predict_proba(test_X)

# Calcular LOG LOSS
log_loss_error = log_loss(test_y, output_test)
print("Log Loss Error:", log_loss_error)

# Calcular ROC AUC
roc_auc_score_error = roc_auc_score(test_y, output_test[:,1])
print("ROC Auc Error:",roc_auc_score_error)

Una forma simple es usar un reporte de clasificación genérico de clasificación que proporciona *Scikit-Learn*.

**Precision**: Es la proporción de *verdaderos positivos* bien clasificados (1) contra el total de casos clasificados como positivos (*verdaderos positivos* más *falsos positivos*). Más cerca a *uno* es mejor, y más cercano a *cero* es peor.

**Recall**: Es la proporción de *verdaderos positivos* bien clasificados (1) contra los casos positivos esperados (*verdaderos positivos* más *falsos negativos*). Más cerca a *uno* es mejor, y más cercano a *cero* es peor.

**Support**: Muestra la cantidad de casos que pertenecen a cada clase.

**Accuracy**: Es la proporción de casos correctamente clasificados (*verdaderos positivos* más *verdaderos negativos*) contra el total de casos. Más cerca a *uno* es mejor, y más cercano a *cero* es peor.

**F1-Score**: Es la media armónica entre la precisión (**precision**) y la sensibilidad (**recall**). Más cerca a *uno* es mejor, y más cercano a *cero* es peor.

In [ ]:
# se calculan el grupo de clasificación
output_test = model.predict(test_X)

# Se calcula el reporte de clasificación
print(classification_report(test_y, output_test))

In [ ]:
param_grid = {'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200]}


model = BayesSearchCV(estimator=model, search_spaces=param_grid, n_iter=1j, scoring='accuracy', cv=3, return_train_score=True, verbose=3)
print(train_X)
model.fit(train_X, train_y)